In [1]:
from pathlib import Path

import analysis
import pandas as pd

In [2]:
LOG_PATH = Path('..', 'log').with_suffix('.jsonl')

In [3]:
files = analysis.get_all_files_matching(
    model_name='/public/hf/models/meta-llama/Meta-Llama-3.1-8B-Instruct/',
    split='answerable',
)

In [4]:
p = Path('..', 'runs', f'{Path(files["filename"][0]).name}')
with p.open('r') as f:
    df = pd.read_json(f, lines=True)

In [ ]:
df = analysis.remove_no_answer_rows(df=df)

In [6]:
df['model_answer'] = df['messages'].apply(analysis.get_model_final_answer)
df['correct'] = df.apply(analysis.check_final_answer, axis=1)

In [7]:
df[['answer', 'model_answer', 'correct']]

,answer,model_answer,correct
0,3.222426e+01,32.2242574604399,True
1,4.728116e+00,4.728116333076132,False
2,3.796131e+10,37961311304.52,True
4,1.414000e+01,14.14,True
6,2.340627e+00,2.3406274391271746,True
...,...,...,...
95,5.392222e+00,5.39,False
96,5.954529e+01,55.071119960256986,False
97,2.800364e+00,2.8003636363636364,False
98,1.000000e+02,100,False


In [8]:
df['gold_tools'] = df['actions'].apply(lambda actions: analysis.get_gold_tool_calls(actions))
df['model_tools'] = df['messages'].apply(lambda messages: analysis.get_model_tool_calls(messages))
df['precision'] = df.apply(lambda row: analysis.precision(row['gold_tools'], row['model_tools']), axis=1)
df['recall'] = df.apply(lambda row: analysis.recall(row['gold_tools'], row['model_tools']), axis=1)
df['accuracy'] = df.apply(lambda row: analysis.accuracy(row['gold_tools'], row['model_tools']), axis=1)

In [9]:
df[['precision', 'recall', 'accuracy']]

,precision,recall,accuracy
0,0.437500,0.777778,0.388889
1,0.750000,0.857143,0.666667
2,0.388889,0.777778,0.350000
4,0.458333,0.846154,0.423077
6,0.791667,0.863636,0.703704
...,...,...,...
95,0.733333,0.846154,0.647059
96,0.360000,0.692308,0.310345
97,0.812500,0.866667,0.722222
98,0.733333,0.846154,0.647059
